In [1]:
from pdf2image import convert_from_bytes
import pytesseract
import os
import pandas as pd
from tqdm import tqdm
import re
import openai
import tiktoken
import random
import json
import fitz
import time
import requests
from requests.packages.urllib3.util import ssl_
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import numpy as np
from collections import defaultdict

In [2]:
os.getcwd()

'C:\\Users\\matia\\OneDrive - Universidad del Pacífico\\01-Medidas_emergencia_PE\\01-DATA_PERU\\03-CODE\\Base principal'

**Set own root directory.**

In [3]:
root = r'C:\Users\matia\OneDrive - Universidad del Pacífico\01-Medidas_emergencia_PE'
os.chdir(root)

In [4]:
data_raw = root + r'\01-DATA_PERU\01-DATA_RAW'
data_pro = root + r'\01-DATA_PERU\02-DATA_PROCESSED'
documentation = root + r'\01-DATA_PERU\04-DATA_DOCUMENTATION'

In [5]:
training_df = pd.read_excel(data_pro + r'\training_dfs\pdf_texts_manual.xlsx')
#training_df = training_df.fillna('')
training_df.drop('extraction_type', axis='columns', inplace=True)
training_df

,text,n_cod_contrato,precio_unitario_1,precio_unitario_2,precio_unitario_3,precio_unitario_4,precio_unitario_5,precio_unitario_6,precio_unitario_7,precio_unitario_8,...,precio_unitario_121,precio_unitario_122,precio_unitario_123,precio_unitario_124,precio_unitario_125,precio_unitario_126,precio_unitario_127,precio_unitario_128,precio_unitario_129,precio_unitario_130
0,"EMAPACOP .A, EMPRESA MUNICIPAL OF AGUA POTABL...",1203413,25378.620,16352.23,10662.0,12431.0,571.0,5344.0,1805.0,23704.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,".. , . ..,.. ,.. . ,,. IAFAS del Ejército del ...",1204449,4.750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2 EMAPACOP S.A. 8 EMPRESA MUNICIPAL DE AGUA PO...,1206365,601.800,8026.95,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sistema Integrado de Gestidn Administrativa Mo...,1206762,3.220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Subasta Inversa Electronica N 0122018CENARES/M...,1207214,2.337,9.49,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,Ministerio SENASA oe stead PROGRAMA DE DESAR...,2071439,0.805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278,PERU de Trabajo ae Sela o Tu ate awe : feet ...,2086854,3.700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
279,isteri Seguro Social de Salud Rex EsSalud y Pr...,2098813,3040000.000,160000.00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280,Sistema Integrado de Gestién Administrativa Mé...,2107267,4.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Keep only the unit price columns
unit_price_columns = [col for col in training_df.columns if 'precio_unitario' in col]

# Function to join non-null values
def join_non_nulls(row):
    non_null_values = [str(value) for value in row if not pd.isnull(value)]
    return '; '.join(non_null_values)

training_df['precios_unitarios'] = training_df[unit_price_columns].apply(join_non_nulls, axis=1)
training_df.drop(unit_price_columns, axis='columns', inplace=True)
training_df

,text,n_cod_contrato,precios_unitarios
0,"EMAPACOP .A, EMPRESA MUNICIPAL OF AGUA POTABL...",1203413,25378.62; 16352.23; 10662.0; 12431.0; 571.0; 5...
1,".. , . ..,.. ,.. . ,,. IAFAS del Ejército del ...",1204449,4.75
2,2 EMAPACOP S.A. 8 EMPRESA MUNICIPAL DE AGUA PO...,1206365,601.8; 8026.95
3,Sistema Integrado de Gestidn Administrativa Mo...,1206762,3.22
4,Subasta Inversa Electronica N 0122018CENARES/M...,1207214,2.337; 9.49
...,...,...,...
277,Ministerio SENASA oe stead PROGRAMA DE DESAR...,2071439,0.805
278,PERU de Trabajo ae Sela o Tu ate awe : feet ...,2086854,3.7
279,isteri Seguro Social de Salud Rex EsSalud y Pr...,2098813,3040000.0; 160000.0
280,Sistema Integrado de Gestién Administrativa Mé...,2107267,4.0


In [7]:
training_df['text'] = training_df['text'].str.lower()
training_df['text'] = training_df['text'].apply(lambda x: re.sub('\s+', ' ', x).strip())
training_df['text'] = training_df['text'].apply(lambda x: re.sub(r'(\.|\,)\1+', r'\1', x).strip())

In [8]:
# Split a text into smaller chunks of size n, preferably ending at the end of a sentence
def create_chunks(text, n, tokenizer):
    tokens = tokenizer.encode(text)
    """Yield successive n-sized chunks from text."""
    i = 0
    while i < len(tokens):
        # Find the nearest end of sentence within a range of 0.9 * n and 1.1 * n tokens
        j = min(i + int(1.1 * n), len(tokens))
        while j > i + int(0.9 * n):
            # Decode the tokens and check for full stop or newline
            chunk = tokenizer.decode(tokens[i:j])
            if chunk.endswith(".") or chunk.endswith("\n"):
                break
            j -= 1
        # If no end of sentence found, use n tokens as the chunk size
        if j == i + int(0.9 * n):
            j = min(i + n, len(tokens))
        yield tokens[i:j]
        i = j

In [9]:
# Initialise tokenizer
tokenizer = tiktoken.encoding_for_model('gpt-3.5-turbo')

prompt=[]

for i in tqdm(range(len(training_df['text']))):
    chunks = create_chunks(training_df.loc[i,'text'], 1800, tokenizer)
    text_chunks = [tokenizer.decode(chunk) for chunk in chunks]
    if len(text_chunks)>=2:
        prompt.append(' '.join(text_chunks[:1]+text_chunks[-1:]))
    else:
        prompt.append(' '.join(text_chunks))
training_df['text'] = prompt

100%|███████████████████████████████████████████████████████████████████████████████| 282/282 [00:02<00:00, 124.38it/s]


# Legacy format

In [10]:
training_df['prompt'] = training_df['text'] + '---->'
training_df['completion'] = ' ' + training_df['precios_unitarios'] + ' \n\n###\n\n'
training_df = training_df.drop(columns = ['text','n_cod_contrato','precios_unitarios'])
training_df

,prompt,completion
0,"emapacop .a, empresa municipal of agua potable...",25378.62; 16352.23; 10662.0; 12431.0; 571.0; ...
1,". , . .,. ,. . ,. iafas del ejército del perú ...",4.75 \n\n###\n\n
2,2 emapacop s.a. 8 empresa municipal de agua po...,601.8; 8026.95 \n\n###\n\n
3,sistema integrado de gestidn administrativa mo...,3.22 \n\n###\n\n
4,subasta inversa electronica n 0122018cenares/m...,2.337; 9.49 \n\n###\n\n
...,...,...
277,ministerio senasa oe stead programa de desarro...,0.805 \n\n###\n\n
278,peru de trabajo ae sela o tu ate awe : feet y ...,3.7 \n\n###\n\n
279,isteri seguro social de salud rex essalud y pr...,3040000.0; 160000.0 \n\n###\n\n
280,sistema integrado de gestién administrativa mé...,4.0 \n\n###\n\n


# Conversational format

In [11]:
output_list = []

for index, row in training_df.iterrows():
    temp_dict = {
        "messages": [
            {"role": "system", "content": "Dado el texto extraido de un contrato, extrae los precios unitarios de los bienes comprados."},
            {"role": "user", "content": row['prompt']},
            {"role": "assistant", "content": str(row['completion'])}
        ]
    }
    output_list.append(temp_dict)

In [13]:
with open(data_pro + r'\training_dfs\training_final.jsonl', 'w') as f:
    for entry in output_list:
        json.dump(entry, f)
        f.write('\n')

In [14]:
data_path = data_pro + r'\training_dfs\training_final.jsonl'

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 282
First example:
{'role': 'system', 'content': 'Dado el texto extraido de un contrato, extrae los precios unitarios de los bienes comprados.'}
{'role': 'user', 'content': 'emapacop .a, empresa municipal of agua potable alcantarillado oe coronel portillo sa. contrato n 001 2019ggemapacop s.a. automatizacion y monitoreo de las fuentes de abastecimiento de agua potable conste por el presente documento, que celebran de una parte la empresa municipal de agua potable y aicantarillado de coronel portillo sociedad anénima, en adelante la entidad, con ruc n 20128985841, con domicilio legal en el jr. julio c. arana n 433 451, debidamente representada por su coordinador rat otass. ing. mirko felix jurado duenas, identificado con dni n 10621651, y de otra parte la empresa esbona corporation sac. con ruc n 20538503755, con domicilio legal en ia av. rivera navarrete n 765 of, 91 cal egy san isidro lima lima, inscrita en la partida electronica n 12590507. del registro de personas juri

In [15]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [16]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [17]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 196, 4032
mean / median: 2428.900709219858, 2416.0
p5 / p95: 1297.9, 3411.9

#### Distribution of num_assistant_tokens_per_example:
min / max: 2, 654
mean / median: 20.421985815602838, 6.0
p5 / p95: 2.0, 41.0

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


In [18]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~684950 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~2054850 tokens


In [20]:
os.environ['OPENAI_API_KEY'] = ""

In [22]:
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.File.create(
  file=open(data_pro + r'\training_dfs\training_final.jsonl', "rb"),
  purpose='fine-tune'
)

<File file id=file-GK4oi4upp7KhYzfP5MbNbm30 at 0x18a2b50b240> JSON: {
  "object": "file",
  "id": "file-GK4oi4upp7KhYzfP5MbNbm30",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 2512994,
  "created_at": 1694214347,
  "status": "uploaded",
  "status_details": null
}

In [26]:
openai.FineTuningJob.create(training_file="file-GK4oi4upp7KhYzfP5MbNbm30", model="gpt-3.5-turbo")

<FineTuningJob fine_tuning.job id=ftjob-APmkr86FJcqQzuxh09Q5xO6W at 0x18a2c708680> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-APmkr86FJcqQzuxh09Q5xO6W",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1694214508,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-Pp6StbgvoyY42pkMUAcanFUD",
  "result_files": [],
  "status": "created",
  "validation_file": null,
  "training_file": "file-GK4oi4upp7KhYzfP5MbNbm30",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null,
  "error": null
}

In [43]:
openai.FineTuningJob.retrieve("ftjob-APmkr86FJcqQzuxh09Q5xO6W")

<FineTuningJob fine_tuning.job id=ftjob-APmkr86FJcqQzuxh09Q5xO6W at 0x18a347399e0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-APmkr86FJcqQzuxh09Q5xO6W",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1694214508,
  "finished_at": 1694216728,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::7wfSDkB5",
  "organization_id": "org-Pp6StbgvoyY42pkMUAcanFUD",
  "result_files": [
    "file-C2Kl5VsouWpKZ5vGwkqs8ib5"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-GK4oi4upp7KhYzfP5MbNbm30",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 2053158,
  "error": null
}